In [121]:
import numpy as np
import tensorflow as tf
from sympy import solve
from sympy import Number, Poly
from sympy.abc import a, b, c, d, e, f, g, I
from sympy.core.add import Add
from sympy.core.symbol import Symbol
from typing import List, Dict
from dataclasses import dataclass, field
from functools import reduce
from IPython.display import display, Image

In [51]:
for expression in [0.5*(a - b) - I, 0.1*(a - d) - I]:
    print(solve(expression, I))
    print(type(solve(expression, I)[0]))

[0.5*a - 0.5*b]
<class 'sympy.core.add.Add'>
[0.1*a - 0.1*d]
<class 'sympy.core.add.Add'>


In [93]:
@dataclass
class CircuitNode:
    label: str
    resistor_exprs: List[Add]
    current_symbol: Symbol
    node_expr: Add = field(default_factory=Add)
    
    def __post_init__(self) -> None:
        has_c_symbol: Dict[int, bool] = {
            idx: le.has(self.current_symbol) for idx, le in enumerate(self.resistor_exprs)
        }
        if not all(has_c_symbol.values()):
            missing_symbol: Dict[int, bool] = {
                idx: test_val for idx, test_val in has_c_symbol.items() if not test_val
            }
            msg: str = f"""
            All resistor expressions must contain the current symbol!
            The following do not:
            {missing_symbol}
            """
            raise ValueError(msg)
            
        current_expressions: List[Add] = CircuitNode.solve_for_current(self.resistor_exprs)
        self.node_expr: Add = CircuitNode.combine_resistor_exprs(current_expressions)

    @staticmethod
    def solve_for_current(resistor_exprs: List[Add]) -> List[Add]:
        out: List[Add] = list(map(lambda le: solve(le, I)[0], resistor_exprs))
        return out
    
    @staticmethod
    def combine_resistor_exprs(resistor_exprs: List[Add]) -> Add:
        out: Add = reduce(lambda first, second: first + second, resistor_exprs)
        return out

x5: CircuitNode = CircuitNode("x5", [0.5*(e - b) - I, 0.1*(e - d) - I, .5*(e - f) - I, .1*(e - 9) - I], I)
    
x5

CircuitNode(label='x5', resistor_exprs=[-I - 0.5*b + 0.5*e, -I - 0.1*d + 0.1*e, -I + 0.5*e - 0.5*f, -I + 0.1*e - 0.9], current_symbol=I, node_expr=-0.5*b - 0.1*d + 1.2*e - 0.5*f - 0.9)

In [28]:
x5.node_expr

-0.5*b - 0.1*d + 1.2*e - 0.5*f - 0.9

In [30]:
class dumb:
    def __init__(self, x):
        self.x = x
        
    def __repr__(self):
        return f"My value is {self.x}"
        
mydumb = dumb(3)
print(mydumb)
        

My value is 3


In [5]:
CircuitNode.solve_for_current(x1.resistor_exprs)

[0.5*a - 0.5*b, 0.1*a - 0.1*d]

In [75]:
x5_expr = x5.node_expr
x5_expr

-0.5*b - 0.1*d + 1.2*e - 0.5*f - 0.9

In [21]:
x5.node_expr.atoms(Number)

{-0.900000000000000, -0.500000000000000, -0.100000000000000, 1.20000000000000}

In [127]:
type(b)

sympy.core.symbol.Symbol

In [128]:
type(a+b)

sympy.core.add.Add

In [47]:
x5.node_expr.coeff(b, 0).coeff(a, 0).coeff(c, 0).coeff(d, 0).coeff(e, 0).coeff(f, 0)

-0.900000000000000

In [54]:
x5.node_expr.collect?

In [125]:
key_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g']

In [126]:
a, b, c, d, e, f, g = map(Symbol, key_list)
key_list

['a', 'b', 'c', 'd', 'e', 'f', 'g']

In [138]:
x5_dict = set(x5.node_expr.as_coefficients_dict().keys())
x5_dict

{1, b, d, e, f}

In [140]:
set1 = {1, 2, 3}

In [141]:
set2 = {2, 3, 4}

In [143]:
set1.union(set2)

{1, 2, 3, 4}

In [111]:
empty_dict = {'a': 0.0, 'b': 0.0, 'c': 0.0, 'd': 0.0, 'e': 0.0, 'f': 0.0, 'g': 0.0}

In [112]:
empty_dict[1] = 0.0
empty_dict

{'a': 0.0, 'b': 0.0, 'c': 0.0, 'd': 0.0, 'e': 0.0, 'f': 0.0, 'g': 0.0, 1: 0.0}

In [113]:
empty_dict.update(x5_dict)
empty_dict

{'a': 0.0,
 'b': 0.0,
 'c': 0.0,
 'd': 0.0,
 'e': 0.0,
 'f': 0.0,
 'g': 0.0,
 1: -0.900000000000000,
 e: 1.20000000000000,
 b: -0.500000000000000,
 f: -0.500000000000000,
 d: -0.100000000000000}

In [100]:
a = [[1, 2, 3, 4], [1, 2, 3, 4]]
a

[[1, 2, 3, 4], [1, 2, 3, 4]]

In [145]:
test_tensor = tf.constant([1, 2])

In [146]:
test_tensor[1]

<tf.Tensor: id=5, shape=(), dtype=int32, numpy=2>

In [149]:
new_tensor = test_tensor + tf.constant([0, 1])
new_tensor

<tf.Tensor: id=9, shape=(2,), dtype=int32, numpy=array([1, 3], dtype=int32)>

In [101]:
b = [r.pop(3) for r in a]
b

[4, 4]

In [102]:
a

[[1, 2, 3], [1, 2, 3]]

In [59]:
def create_keys(input_circuit: List[int]) -> List:
        """creates an alphabetical list of keys"""
        number_of_keys: int = len(input_circuit)
        key_list: List = []
        alpha = 'a'
        for i in range(0, number_of_keys):
            key_list.append(alpha)
            alpha = chr(ord(alpha) + 1)
        return key_list

In [69]:
x: List[int] = [1, 2, 3, 4, 5, 6, 7]

In [74]:
x_c = create_keys(x)
x_c

['a', 'b', 'c', 'd', 'e', 'f', 'g']

In [79]:
def find_constants(input_expr: Add, key_list: List) -> float:
    """finds the constant value in a node expression"""
    output_number: float = 0.0
    for i in range(0, len(key_list)):
        x = key_list[i]
        output_number = input_expr.coeff(x, 0)
    return 0.0 - output_number

In [80]:
find_constants(x5_expr, x_c)

0.5*b + 0.1*d - 1.2*e + 0.5*f + 0.9

In [169]:
(lambda a,b: a.update(b) or a)({'a':1, 'b':3},{'c':5})

{'a': 1, 'b': 3, 'c': 5}

In [168]:
type((lambda a,b: a.update(b) or a)({'a':1, 'b':3},{'c':5}))

dict

In [171]:
type((lambda a,b: a.update(b))({'a':1, 'b':3},{'c':5}))

NoneType